In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver  
import pandas as pd

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Start the browser
driver = webdriver.Chrome()

# Open the URL
# انشطه
url = 'https://riyadh.sa/ar/events/all'
driver.get(url)

 # extract the body and parse it
soup = BeautifulSoup(driver.page_source, "html.parser")
#<div rd-dot="rd-dot" ng-class="{'-lg' : event.featured == true}" class="rd-entry-item-img-title-text ng-binding" style="overflow-wrap: break-word;">سوق جاكس الرمضاني</div>
    # extrace prices and ars names from page
Names = soup.find_all('div', {'class':"rd-entry-item-img-title-text ng-binding"})


In [4]:
len(Names)

12

In [5]:
Names

[<div class="rd-entry-item-img-title-text ng-binding" ng-class="{'-lg' : event.featured == true}" rd-dot="rd-dot" style="overflow-wrap: break-word;">سوق جاكس الرمضاني</div>,
 <div class="rd-entry-item-img-title-text ng-binding" ng-class="{'-lg' : event.featured == true}" rd-dot="rd-dot" style="overflow-wrap: break-word;">فعالية "بازار ريف"</div>,
 <div class="rd-entry-item-img-title-text ng-binding" ng-class="{'-lg' : event.featured == true}" rd-dot="rd-dot" style="overflow-wrap: break-word;">ليالي كافد الرمضانية</div>,
 <div class="rd-entry-item-img-title-text ng-binding" ng-class="{'-lg' : event.featured == true}" rd-dot="rd-dot" style="overflow-wrap: break-word;">تحرك معنا 2025 – الرياض</div>,
 <div class="rd-entry-item-img-title-text ng-binding" ng-class="{'-lg' : event.featured == true}" rd-dot="rd-dot" style="overflow-wrap: break-word;">بطولة البلوت</div>,
 <div class="rd-entry-item-img-title-text ng-binding" ng-class="{'-lg' : event.featured == true}" rd-dot="rd-dot" style="over

# Get Urls links form webook

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# Initialize the WebDriver
driver = webdriver.Chrome()

# Base URL
url = "https://riyadh.sa/ar/events/all"

# Open the webpage
driver.get(url)

# Notify the user
print("The page has loaded. You can now scroll and click the 'Load More' button manually if needed.")

# Pause execution until the user is ready
input("Press Enter to continue with scraping...")

# Extract the page source after manual interaction
html = driver.page_source

# Initialize an empty list to store the URLs
url_list_evants = []

# Function to extract event URLs
def extract_event_urls():
    try:
        # Wait for the visibility of the div containing the event links
        WebDriverWait(driver, 50).until(
            EC.visibility_of_all_elements_located((By.XPATH, "//div[contains(@class, 'rd-entry-item-inner')]//a[@href]"))
        )
    except TimeoutException:
        print("Timeout: The elements did not appear in time!")
        return

    # Find all anchor tags within rd-entry-item-inner divs
    elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'rd-entry-item-inner')]//a[@href]")

    for element in elements:
        event_url = element.get_attribute("href")
        if event_url and "riyadh.sa/ar/events/all/item/event" in event_url:
            # Append to the list if the URL matches the event pattern
            if event_url not in url_list_evants:
                url_list_evants.append(event_url)

# Extract event URLs from the current page
extract_event_urls()

# Print the collected event URLs
print("Collected Event URLs:", url_list_evants)

# Close the WebDriver
driver.quit()


The page has loaded. You can now scroll and click the 'Load More' button manually if needed.
Timeout: The elements did not appear in time!
Collected Event URLs: []


In [10]:
len(set(url_list_evants))

42

In [ ]:
url_series = pd.Series(url_list_evants)

# Save the Series to a CSV file
url_series.to_csv('Data/urls_data/url_list_evants_riyadh.csv', index=False, header=False)

# Get the Data from Urls

In [28]:
# Initialize an empty DataFrame for storing the results
event_data = pd.DataFrame(columns=['names', 'price', 'date', 'location', 'description', 'location_url', 'type'])

In [33]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Initialize the WebDriver
driver = webdriver.Chrome()

# Read the CSV file containing URLs (assuming the CSV file has one column with URLs)
urls_activitesc = pd.read_csv('url_data/url_list_evants_riyadh.csv', header=None)  # No header for one-column CSV

# Rename the column for easier access
urls_activitesc.columns = ['url']

# Add a new column 'type' with the value 'Exhibitions and festivals' for all rows
urls_activitesc['type'] = 'Exhibitions and festivals'

# Initialize an empty DataFrame to store the scraped data
event_data = pd.DataFrame(columns=['names', 'price', 'date', 'location', 'description', 'location_url', 'url', 'type'])

# Loop through each URL in the URLs from the CSV file
for index, row in urls_activitesc.iterrows():
    url = row['url']  # Access the URL
    
    # Request the page
    driver.get(url)

    try:
        # Wait for the page body to load instead of waiting for a specific element
        WebDriverWait(driver, 40).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
    except:
        print(f"Timeout occurred while loading the page: {url}")
        continue
    
    # Print the page source to verify the elements
    page_source = driver.page_source
    print(f"Page source for {url}:")
    print(page_source)  # Output page source to see if the elements are present
    
    # Parse the page with BeautifulSoup
    soup = BeautifulSoup(page_source, "html.parser")

    # Extract event name from the correct <h1> inside the <div class="rd-article-header -lg">
    name_div = soup.find('div', class_='rd-article-header -lg').find('h1', class_='ng-binding') if soup.find('div', class_='rd-article-header -lg') else None
    name = name_div.text.strip() if name_div else 'N/A'
    print(f"Event Name: {name}")  # Print name to verify extraction
    
    # Extract price (set as null for now)
    price = 'null'

    # Extract date (two <p> elements in the 'rd-article-info-text' class)
    date_div = soup.find('div', class_='rd-article-info-text')
    date = 'N/A'
    if date_div:
        date_p1 = date_div.find_all('p')
        if len(date_p1) > 0:
            date = date_p1[0].text.strip()  # First <p> for date range
        if len(date_p1) > 1:
            date += ' ' + date_p1[1].text.strip()  # Second <p> for time range
    print(f"Event Date: {date}")  # Print date to verify extraction

    # Extract location and location URL
    location_div = soup.find('div', class_='rd-article-adress-text')
    location_text = 'N/A'
    location_url = 'N/A'
    if location_div:
        location_a = location_div.find('a')
        if location_a:
            location_text = location_a.text.strip()
            location_url = location_a['href']
    print(f"Event Location: {location_text}")  # Print location to verify extraction

    # Extract description (first <p> in the rd-article-text class)
    description_div = soup.find('div', class_='rd-article-text ng-binding')
    description = 'N/A'
    if description_div:
        description_p = description_div.find('p')
        description = description_p.text.strip() if description_p else 'N/A'
    print(f"Event Description: {description}")  # Print description to verify extraction

    # Extract the website URL
    website_div = soup.find('div', class_='rd-article-info-col ng-scope')
    website_url = 'N/A'
    if website_div:
        website_a = website_div.find('a', class_='rd-article-info-link ng-binding ng-scope')
        if website_a and website_a.has_attr('href'):
            website_url = website_a['href']  # Extract the URL from the 'href' attribute
    print(f"Website URL: {website_url}")  # Print website to verify extraction

    # Store the extracted data in a DataFrame
    new_row = pd.DataFrame({
        'names': [name],
        'price': [price],
        'date': [date],
        'location': [location_text],
        'description': [description],
        'location_url': [location_url],
        'url': [website_url],  # Add the new 'url' column
        'type': ['Exhibitions and festivals']  # Add the type column with value 'Exhibitions and festivals'
    })

    # Concatenate with the main DataFrame
    event_data = pd.concat([event_data, new_row], ignore_index=True)
    break



# Close the WebDriver
driver.quit()


Page source for https://riyadh.sa/ar/events/all/item/event/28027:
<html lang="ar" class="no-js"><head><meta http-equiv="origin-trial" content="A/kargTFyk8MR5ueravczef/wIlTkbVk1qXQesp39nV+xNECPdLBVeYffxrM8TmZT6RArWGQVCJ0LRivD7glcAUAAACQeyJvcmlnaW4iOiJodHRwczovL2dvb2dsZS5jb206NDQzIiwiZmVhdHVyZSI6IkRpc2FibGVUaGlyZFBhcnR5U3RvcmFnZVBhcnRpdGlvbmluZzIiLCJleHBpcnkiOjE3NDIzNDIzOTksImlzU3ViZG9tYWluIjp0cnVlLCJpc1RoaXJkUGFydHkiOnRydWV9"><style type="text/css">@charset "UTF-8";[ng\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide:not(.ng-hide-animate){display:none !important;}ng\:form{display:block;}.ng-animate-shim{visibility:hidden;}.ng-anchor{position:absolute;}</style><meta charset="utf-8"><title ng-bind-html="siteTitle" class="ng-binding">فعاليات الرياض</title><meta name="google-site-verification" content="poDFxQUR5aHX90eQxgP-TXWL8u11D212cUdokzxeE8c"><link rel="icon" type="image/gif" href="/assets/img/favicon-16x16.png"><link rel="icon" type="image/gif" href="/asse

In [32]:
event_data

,names,price,date,location,description,location_url,url,type
0,N/A,null,N/A,N/A,N/A,N/A,N/A,Exhibitions and festivals
